In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm
import sklearn

**Explore data distribution and remove outliers**

In [70]:
# -- Read matrix containing embeddings and labels
path_df = '/Users/leandro/Desktop/ai_data/X.csv' 
df = pd.read_csv(path_X)

In [71]:
# Remove undesired column and filter mammalian entries
df = df.drop(columns=['Unnamed: 0'])
df_mammals = df[df['Taxonomy'] == 'Mammalia']

**Check how CN and prot_len distribution changes when removing non-mammals**

In [ ]:
X['Non_redundant'].hist(bins = 20, figsize=(8,6))
#plt.xlabel('Non Redundant')
#plt.ylabel('copy number')
plt.title('Complete dataset')

In [ ]:
X_mammals['Non_redundant'].hist(bins = 20, figsize=(8,6))
plt.title('Mammalian sequences')

In [ ]:
X['prot_len'].hist(bins = 20, figsize=(8,6))
plt.title('Complete dataset')

In [ ]:
X_mammals['prot_len'].hist(bins = 20, figsize=(8,6))
plt.title('Mammalian sequences')

**Explore correlation among all dimensions**

In [ ]:
# -- Compute correlation matrix among dimensiones (contains Pearson's r)
corr_matrix = X_mammals.corr() # generate correlation matrix
correlations = corr_matrix['Non_redundant'].sort_values(ascending=False) # check columns (dimensions) that correlates more to CN

In [ ]:
# -- Different to plot correlation matrix
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
plt.matshow(corr)
cb = plt.colorbar()
plt.xlabel('dimension')
plt.ylabel('dimension')
plt.title('Correlation Matrix', fontsize=12)

In [ ]:
# -- This explores pairwise correlation (what kind?) among ALL DIMENSIONS (not only against CN)
split_df = X_mammals.iloc[:9,:10]   # generate a subset from the original dataset
corr = df.corr()   # Compute correlaion (based on Pearson)

In [ ]:
# -- Exploring another correlation coefficient matrix with Numpy
numbers = split_df.iloc[:,1:]  # select only numerical values 
cm = np.corrcoef(numbers.values.T)  # Compute correlation coefficient
sns.set(font_scale=1) # letter size
hm = sns.heatmap(cm, cmap='coolwarm', cbar=True, annot=True, square=True, fmt = '.2f', annot_kws={'size': 10},) # yticklabels=cols, xticklabels=cols

# Explore correlation between each vector dimension and CN values

In [ ]:
X_mammals_numbers = X_mammals.iloc[:,1:-2]  # leave only embs dimensions and copy number values
corr_matrix = X_mammals_numbers.corr() # compute correlation matrix
correlations_cn = corr_matrix['Non_redundant'].sort_values(ascending=False) # check columns (dimensions) that correlates more to CN

In [ ]:
correlations = correlations_cn.iloc[1:]  # remove first row that match CN againts itself (coefficient = 1)
plt.scatter(y=correlations.values, x=correlations.index) # Plot correlation variation (sorted)
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.ylabel('Pearsons correlation coefficient against CN')
plt.xlabel('dimensions')
plt.title('All dimensions')

In [ ]:
# -- Convert pandas series to dataframe for easier manipulation
a = pd.DataFrame(correlations)
b = a.reset_index()
#correlations_df.columns # display columns names
correlations_df = b.rename(columns={'index':'dimension-1','Non_redundant':'Pearson'})

In [ ]:
# -- Filter only positive values to look for the elbow of the curve
positive_corre = correlations_df[correlations_df.Pearson >= 0] # select only positive values
highest_corre = correlations_df[correlations_df.Pearson >= 0.3] # select values >= 0.3

In [ ]:
plt.scatter(y=positive_corre['Pearson'].values, x=positive_corre['dimension-1'].values)
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.ylabel('Pearsons correlation coefficient against CN')
plt.xlabel('dimensions')
plt.title('Positive correlation')

In [ ]:
plt.scatter(y=highest_corre['Pearson'].values, x=highest_corre['dimension-1'].values)
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.ylabel('Pearsons correlation coefficient against CN')
plt.xlabel('dimensions')
plt.title('Dimensions with Pearsons >0.3')

In [ ]:
# -- Leave only 10 most significant embeddings
X_elbow = X_mammals.filter(items=['ID','988','145','265','933','113','823','266','777','56','562','Non_redundant'])
X_elbow

# Selecting important features using random forest

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

**Cookbook approach**

In [ ]:
features = X_mammals_numbers.iloc[:,0:-1]
target = X_mammals_numbers.iloc[:,-1]

In [ ]:
# Create random forest classifier
forest = RandomForestClassifier(random_state=0, n_jobs=-1)
# Create object that selects features with importance greater than or equal to a thresold
selector = SelectFromModel(forest, threshold=0.2)

In [ ]:
# New feature matrix using selector
features_important = selector.fit_transform(features, target)

In [ ]:
# Train random forest using most important features
model = forest.fit(features_important,target)

**Book approach**

In [77]:
# -- Define X and y from whole dataset and split
X = df.iloc[:,1:-3]
y = df['Non_redundant'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 0)  

In [79]:
# Define feature labels (vector dimensions in this case)
feat_labels = X.columns

In [18]:
# Create random forest classifier
forest = RandomForestClassifier(random_state=1)
forest.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [80]:
importances = forest.feature_importances_  #return importance for every dimension
indices = np.argsort(importances[::-1])   # np.argsort returns the indices that would sort an array. 

In [81]:
np.argsort(importances)

array([ 663,  945,  685, ...,  679,  524, 1015])

In [82]:
np.argsort(importances[::-1])

array([360,  78, 338, ..., 344, 499,   8])

In [48]:
test_array = [0,1,2,3,4]
test_array[::-1]

[4, 3, 2, 1, 0]

In [65]:
np.argsort(test_array)

array([0, 1, 2, 3, 4])

In [66]:
np.argsort(test_array[::-1])

array([4, 3, 2, 1, 0])

In [ ]:
# Show features importances 
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

In [ ]:
# Plot features importances 
plt.title('Feature importance')
plt.bar(range(X_train.shape[1]), importances[indices], align='center')
plt.xticks(range(X_train.shape[1], feat_labels, rotation=90))
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout

# Train different models using highest-scored dimensions

In [ ]:
def plot_residuals(y_train_pred, y_train, y_test_pred, y_test):
    """Define function to plot residuals"""
    plt.scatter(y_train_pred, y_train_pred - y_train, c='steelblue', marker='o', 
                edgecolor='white', label='Training data')
    plt.scatter(y_test_pred, y_test_pred - y_test, c='limegreen', marker='s', 
                edgecolor='white', label='Test data')
    plt.xlabel('Predicted values')
    plt.ylabel('Residuals')
    plt.legend(loc='upper left')
    plt.hlines(y=0, xmin=0, xmax=max(y_train_pred), color='black', lw=2) 
    plt.xlim([0, max(y_train_pred)])

**Simple linear regression**

In [ ]:
# -- Define X and y from X_elbow df, and split
X = X_elbow.iloc[:,1:-1].values
y = X_elbow['Non_redundant'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 0)  

In [ ]:
slr = LinearRegression()  # Define simple linear regression (LR)

In [ ]:
slr.fit(X_train, y_train) # fit model to training data 
y_train_pred = slr.predict(X_train)  # Relate y to X in training data
y_test_pred = slr.predict(X_test) # Predict y_test from X_test

In [ ]:
# -- Evaluate model performance using MSE and R^2 (a standardized version of MSE with better interpretability)
print('MSE -- train: %.3f, test: % .3f' %
      (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)),
      '\nR^2 -- train: %.3f, test: %.3f' %
      (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))

In [ ]:
plot_residuals(y_train_pred, y_train, y_test_pred, y_test)

**LASSO regularization**

In [ ]:
# -- Define X and y from X_elbow df, and split
X = X_elbow.iloc[:,1:-1].values
y = X_elbow['Non_redundant'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 0)  

In [ ]:
lasso = linear_model.Lasso(alpha=0.1)

In [ ]:
lasso.fit(X, y)

In [ ]:
lasso.score(X,y)

**Random Forest regression**

In [ ]:
forest = RandomForestRegressor(n_estimators=100, criterion='squared_error', random_state=1, n_jobs=-1, max_depth=15)

In [ ]:
forest.fit(X_train, y_train)


In [ ]:
y_train_pred = forest.predict(X_train)

In [ ]:
y_test_pred = forest.predict(X_test)

In [ ]:
# -- Evaluate model performance using MSE and R^2 (a standardized version of MSE with better interpretability)
print('MSE -- train: %.3f, test: % .3f' %
      (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)),
      '\nR^2 -- train: %.3f, test: %.3f' %
      (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))

In [ ]:
plot_residuals(y_train_pred, y_train, y_test_pred, y_test)

**Robust regression model using RANSAC**

In [ ]:
ransac = RANSACRegressor(LinearRegression(), max_trials=100, min_samples=50, 
                         loss='squared_error', residual_threshold=5.0, random_state=0)

In [ ]:
y = y.reshape(-1,1)

In [ ]:
X.shape

In [ ]:
ransac.fit(X,y)

In [ ]:
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)
line_X = np.arange(0,max(y),1)
line_y_ransac = ransac.predict(line_X[:, np.newaxis])
plt.scatter(X[inlier_mask], y[inlier_mask], c='steelblue', edgecolor='white', marker='o', label='Inliers')
plt.scatter(X[outlier_mask_mask], y[outlier_mask], c='limegreen', edgecolor='white', marker='s', label='Outliers')
plt.plot(line_X, line_y_ransac, color='black', lw=2)
plt.xlabel('Copy number(M)')
plt.ylabel('dimension')
plt.legend(loc='upper left')

In [ ]:
X.shape

# Training simple linear regression with whole vector to compare

In [ ]:
# -- Define X and y from X_mammals df, and split
X = X_mammals.iloc[:,1:-3]
y = X_mammals['Non_redundant'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 0)  

In [ ]:
slr.fit(X_train, y_train) # fit model to training data 
y_train_pred = slr.predict(X_train)  # Relate y to X in training data
y_test_pred = slr.predict(X_test) # Predict y_test from X_test

In [ ]:
# -- Evaluate model performance using MSE and R^2 (a standardized version of MSE with better interpretability)
print('MSE -- train: %.3f, test: % .3f' %
      (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)),
      '\nR^2 -- train: %.3f, test: %.3f' %
      (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))

In [ ]:
plot_residuals(y_train_pred, y_train, y_test_pred, y_test)